In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os
from getpass import getpass
from dotenv import load_dotenv


load_dotenv("../.env")

def _getpass(env_var: str):
    if not os.environ.get(env_var):
        os.environ[env_var] = getpass(f"{env_var}=")


_getpass("OPENAI_API_KEY")

In [29]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from prompts import initial_design_prompt
from schemas import InitialSystemDesignOutput
from task_flow_inputs import project_description, requirements, user_stories


model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.5)

parser = JsonOutputParser(pydantic_object=InitialSystemDesignOutput)
prompt = ChatPromptTemplate.from_template(
    initial_design_prompt,
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

chain = prompt | model | parser
output = chain.invoke({"project_description": project_description, "requirements": requirements, "user_stories": user_stories})
output


/Users/sebastian/repos/agency/project-estimator/.venv/lib/python3.13/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='A description of the architecture design choices made and the reasons behind them.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/sebastian/repos/agency/project-estimator/.venv/lib/python3.13/site-packages/pydantic/json_schema.py:2279: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='Narrative description of the overall system, its components and their interactions. The components should be in CAPITAL LETTERS.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/sebastian/repos/agency/project-estimator/.venv/lib/python3.13/sit

{'architecture_design': 'Microservices Architecture',
 'component_description': 'The system is designed using a microservices architecture to enhance scalability, maintainability, and deployment flexibility.',
 'interfaces_between_components': 'Components will communicate through RESTful APIs and message queues to ensure loose coupling and asynchronous processing where necessary.',
 'architecture_choices': [{'id': 'AC-1',
   'description': 'Microservices architecture allows for independent deployment and scaling of components.',
   'rationale': 'This choice supports the need for rapid iteration and scaling as the user base grows.'},
  {'id': 'AC-2',
   'description': 'Use of role-based access control (RBAC) for user management.',
   'rationale': 'This addresses FR-1.2 and ensures that permissions are managed effectively based on user roles.'},
  {'id': 'AC-3',
   'description': 'Implementation of a responsive web interface using modern front-end frameworks.',
   'rationale': 'This meet

In [24]:
{'architecture_design': 'The architecture of TaskFlow will follow a microservices-oriented design, where each core functionality (User Management, Project Management, Task Management, Team Collaboration, Reporting) is encapsulated within its own service. This allows for easier deployments, scalability, and independent updates for each component, while also aligning well with the expectation to support up to 100 concurrent users (NFR-1.1). Each service will be containerized using Docker, facilitating consistent environments across development and production. The services will communicate over a REST API framework and a message broker (like RabbitMQ) for asynchronous tasks like notifications and report generation.',

 'component_description': '1. User Management Service: Handles user registration, authentication, and profile management (FR-1.1). It integrates with an identity provider for secure authentication and implements role-based access control (FR-1.2).\n\n2. Project Management Service: Manages project creation, updates, and archiving (FR-2.1), along with handling deadlines, descriptions, and team assignments (FR-2.2).\n\n3. Task Management Service: Responsible for task creation, assignment, and tracking (FR-3.1). It also handles task priorities, deadlines, attachments, and status updates (FR-3.2).\n\n4. Team Collaboration Service: Facilitates commenting on tasks and projects (FR-4.1), file sharing (FR-4.2), and sending notifications for updates and deadlines (FR-4.3).\n\n5. Reporting Service: Generates progress reports for projects and tasks (FR-5.1) and provides analytics on team and individual productivity (FR-5.2).',
 
 'interfaces_between_components': 'Components will interact through well-defined RESTful APIs. The User Management Service will provide endpoints for user data and role checks. The Project Management Service will interact with the Task Management Service to retrieve task assignments and status within projects. The Team Collaboration Service will subscribe to updates from both the Task and Project Management Services to provide real-time notifications and comments. The Reporting Service will utilize data from the Project and Task Management Services to compile reports. Message queues like RabbitMQ can be used for events like notifying users of new tasks or updates, enabling more reliable asynchronous processing.'}

print(model)

client=<openai.resources.chat.completions.completions.Completions object at 0x118b23650> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x118b8e350> root_client=<openai.OpenAI object at 0x118aeb610> root_async_client=<openai.AsyncOpenAI object at 0x118aebd90> model_name='gpt-4o-mini' model_kwargs={} openai_api_key=SecretStr('**********')


In [ ]:
{'architecture_design': 'Microservices Architecture',
 'component_description': 'The architecture is designed to facilitate scalability and ease of maintenance, reflecting the core functionalities of TaskFlow.',
 'interfaces_between_components': 'REST APIs will be used for inter-component communication and data exchange, ensuring clear separations and easy integration.',
 'architecture_choices': [{'id': 'AC-1',
   'description': 'Microservices allow for independent deployment and scaling, which is beneficial for a platform with potentially varying loads across departments.',
   'rationale': "This choice supports the project's goal of enhancing productivity by allowing teams to work on different components simultaneously without interference."},
  {'id': 'AC-2',
   'description': 'Using a relational database for user and task management ensures strong consistency and integrity of transactional data.',
   'rationale': 'TaskFlow requires reliable data handling, especially for user management (FR-1) and task management (FR-3). A relational database is suitable for this.'},
  {'id': 'AC-3',
   'description': 'Choosing a cloud-hosting provider for deployment allows for flexibility in scaling resources based on user demand.',
   'rationale': 'This aligns with performance requirements (NFR-1.1) and ensures the system can handle up to 100 concurrent users efficiently.'}],
 'components': [{'name': 'USER MANAGEMENT',
   'description': 'Handles all aspects of user registration, authentication, profile management, and role-based permissions.',
   'technologies': ['OAuth 2.0 for authentication',
    'JWT for session management',
    'PostgreSQL or MySQL for user data storage']},
  {'name': 'PROJECT MANAGEMENT',
   'description': 'Responsible for creating, updating, and archiving projects, along with their details such as deadlines, descriptions, and team assignments.',
   'technologies': ['Spring Boot or Express.js for backend service',
    'PostgreSQL or MySQL for project data storage']},
  {'name': 'TASK MANAGEMENT',
   'description': 'Manages tasks within projects, allowing creation, assignment, tracking, and support for priorities, deadlines, and attachments.',
   'technologies': ['Spring Boot or Express.js for backend service',
    'PostgreSQL or MySQL for task data storage']},
  {'name': 'COLLABORATION',
   'description': 'Facilitates team communication through comment threads on tasks and projects, as well as file sharing and notifications for updates.',
   'technologies': ['WebSockets for real-time notifications',
    'AWS S3 or similar for file storage']},
  {'name': 'REPORTING',
   'description': 'Generates progress reports and analytics on productivity for teams and individual users.',
   'technologies': ['Python or R for data analysis',
    'Power BI or Tableau for reporting visualizations']}],
 'component_links': [{'first_component': 'USER MANAGEMENT',
   'second_component': 'PROJECT MANAGEMENT',
   'description': 'User roles and permissions are required to access and manage projects.'},
  {'first_component': 'PROJECT MANAGEMENT',
   'second_component': 'TASK MANAGEMENT',
   'description': 'Projects are the container for tasks, and operations on tasks relate directly to their associated project.'},
  {'first_component': 'TASK MANAGEMENT',
   'second_component': 'COLLABORATION',
   'description': 'Each task has associated comments and file attachments that enhance communication.'}]}

In [ ]:
{'architecture_design': 'Microservices Architecture',
 'component_description': 'The system is designed using a microservices architecture to enhance scalability, maintainability, and deployment flexibility.',
 'interfaces_between_components': 'Components will communicate through RESTful APIs and message queues to ensure loose coupling and asynchronous processing where necessary.',
 'architecture_choices': [{'id': 'AC-1',
   'description': 'Microservices architecture allows for independent deployment and scaling of components.',
   'rationale': 'This choice supports the need for rapid iteration and scaling as the user base grows.'},
  {'id': 'AC-2',
   'description': 'Use of role-based access control (RBAC) for user management.',
   'rationale': 'This addresses FR-1.2 and ensures that permissions are managed effectively based on user roles.'},
  {'id': 'AC-3',
   'description': 'Implementation of a responsive web interface using modern front-end frameworks.',
   'rationale': 'This meets NFR-4.1 and ensures usability across devices, enhancing user experience.'},
  {'id': 'AC-4',
   'description': 'Data encryption in transit and at rest.',
   'rationale': 'This fulfills NFR-3.1 and ensures that user data is protected against unauthorized access.'},
  {'id': 'AC-5',
   'description': 'Daily backups of all data.',
   'rationale': 'This meets NFR-2.2 and ensures data recovery in case of failures.'}],
 'components': [{'name': 'USER MANAGEMENT',
   'description': 'Handles user registration, authentication, and profile management, including role-based permissions.',
   'technologies': ['Node.js', 'Express', 'MongoDB']},
  {'name': 'PROJECT MANAGEMENT',
   'description': 'Manages projects, including creation, updates, and archiving of project details.',
   'technologies': ['Node.js', 'Express', 'MongoDB']},
  {'name': 'TASK MANAGEMENT',
   'description': 'Facilitates task creation, assignment, tracking, and management of task-related features.',
   'technologies': ['Node.js', 'Express', 'MongoDB']},
  {'name': 'TEAM COLLABORATION',
   'description': 'Provides features for commenting, file sharing, and notifications related to tasks and projects.',
   'technologies': ['Node.js', 'Express', 'Socket.io']},
  {'name': 'REPORTING',
   'description': 'Generates progress reports and analytics on productivity for projects and tasks.',
   'technologies': ['Node.js', 'Express', 'Chart.js']},
  {'name': 'FRONT-END',
   'description': 'User interface for the application, allowing users to interact with the system in a responsive manner.',
   'technologies': ['React', 'Redux', 'CSS']}],
 'component_links': [{'first_component': 'USER MANAGEMENT',
   'second_component': 'PROJECT MANAGEMENT',
   'description': 'User management provides authentication and authorization for project management actions.'},
  {'first_component': 'PROJECT MANAGEMENT',
   'second_component': 'TASK MANAGEMENT',
   'description': 'Projects serve as containers for tasks, allowing for task management under specific projects.'},
  {'first_component': 'TASK MANAGEMENT',
   'second_component': 'TEAM COLLABORATION',
   'description': 'Tasks include collaboration features such as comments and file attachments.'},
  {'first_component': 'PROJECT MANAGEMENT',
   'second_component': 'REPORTING',
   'description': 'Reporting component pulls data from project management to generate progress reports.'},
  {'first_component': 'FRONT-END',
   'second_component': 'USER MANAGEMENT',
   'description': 'The front-end interacts with the user management component for authentication and profile management.'}]}